In [ ]:
# Libraries

#Standard libraries of Python
import sys
sys.path.append('C:\Proyectos\Loteria\database')
sys.path.append('C:\Proyectos\Loteria\data_analisys')
import datetime
import time
from collections import Counter
from decimal import Decimal, ROUND_HALF_UP, getcontext
getcontext().prec = 5

# Dependencies
import pandas as pd
import numpy as np
np.set_printoptions(precision=5)

# Libraries made for this proyect
import data_functions

In [ ]:
"""Constants of the main code"""

# Main Database
db = pd.read_parquet('C:\Proyectos\Loteria\DataBase\db.parquet')
db_slice = db.head(13)
winning_numbers = db_slice.iloc[:, 2:7]

# Array of numbers
total_numbers = np.arange(1,51)

# Array of stars
total_stars = np.arange(1,13)

# Extra row, in order to compare the number that did not appear for the first time in the game history
d_0 = pd.DataFrame(columns=[str(i) for i in range(1,51)], index=[0]).fillna(True)

# Create a template DataFrame with all values set to False
skip_winners_bool = pd.DataFrame(False, columns=[str(i) for i in range(1,51)], index=range(len(winning_numbers)))

# Array of the number of skips per game
skips = np.arange(0,19)

In [ ]:
main_df_counts = data_functions.first_df_bool(db, total_numbers) # execute just once

In [ ]:
# Fill in the True values
for e in range(1,6):
    col_name = f"Nro{e}"
    skip_winners_bool = skip_winners_bool | (winning_numbers[col_name].to_numpy()[:, None] == total_numbers)
skip_winners_bool = pd.concat([d_0, skip_winners_bool]).reset_index(drop=True)
    
# Year History for numbers and stars
numbers_year_history, total_hits = data_functions.year_hits(db, winning_numbers, total_numbers, data_functions.count_hits)
ny_mean = pd.DataFrame(numbers_year_history.mean(), columns=['Average']).T.rename(index={'0': 'Average'})
ny_median = pd.DataFrame(numbers_year_history.median(), columns=['Median']).T.rename(index={'0': 'Median'}).applymap(lambda x:(int(x+0.5)))
numbers_year_history = pd.concat([ny_median, ny_mean, numbers_year_history])

# Average of hits per numbers
numbers_average = data_functions.average_hits(db, total_hits, total_numbers)

# Get the natural rotation of the numbers
df_aprox, df_exact = data_functions.get_rotations(db, total_hits, total_numbers, numbers_average, is_star=False)

In [ ]:
# Continue of the Main Counts Df
mcd_last_row = main_df_counts.iloc[-1:].reset_index(drop=True)

# specific row of Boolean Df in Current Flow
bdcf_last_row = skip_winners_bool.iloc[len(db_slice), :].to_frame().T.reset_index(drop=True)

# Replace the values of last row to prepare for sum
true_to_nan = bdcf_last_row.replace(True, np.nan)
bdcf_last_row = true_to_nan.replace(False, 1)

# Sum of rows
new_row = mcd_last_row + bdcf_last_row
new_row = new_row.replace(np.nan, 0).astype(int)

# New version of Main Counts Df
main_df_count = pd.concat([main_df_counts, new_row], ignore_index=True)
main_df_count.index = main_df_count.index + 1

In [ ]:
main_df_count

In [ ]:
last_draw = new_row.transpose().sort_values(by=0).reset_index()
last_draw = last_draw.rename(columns={'index': 'Numero', 0: 'Skips'})

# Select the last 12 draws
last_12_draws = np.arange(len(main_df_count) - 11, len(main_df_count) + 1)
sk_12 = main_df_count.loc[last_12_draws]

# This establish the skips of the last 12 draws
aus_12 = [sk_12.loc[i, str(column)] for i in last_12_draws[0:11] for column in sk_12 if sk_12.loc[i, str(column)] == 0]
counter_7 = Counter(aus_12[25:60])
counter_12 = Counter(aus_12)
last_7 = [counter_7.get(i,0) for i in skips]
last_12 = [counter_12.get(i,0) for i in skips]
skips_7_12 = pd.DataFrame({'7': last_7, '12': last_12})

In [ ]:
len(aus_12)

In [ ]:
groups = [list(range(i,i+10)) for i in range(1,51,10)]
group_names = [tuple(range(i,i+10)) for i in range(1,51,10)]
results = {i: {group_name: sum([1 for num in row if num in group]) for group_name, group in zip(group_names, groups)} for i, row in winning_numbers.iterrows()}
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.columns = [f'{i}_to_{i+9}' for i in range(1,51,10)]
future_sg_10 = results_df.iloc[-9:]
future_sg_5 = results_df.iloc[-4:]
future_groups_df = pd.DataFrame({'10_games': (future_sg_10 > 0).sum(), '5_games': (future_sg_5 > 0).sum()}).T

In [ ]:
years = db['Dates'].dt.year.to_frame()
years = years.rename(columns={'Dates': 'Years'})
months = db['Dates'].dt.month.to_frame()
months = months.rename(columns={'Dates': 'Month'})
year_month = pd.concat([years, months, winning_numbers], axis=1)
series_dates = year_month.melt(id_vars=['Years', 'Month'], var_name='Data', value_vars=['Nro1', 'Nro2', 'Nro3', 'Nro4', 'Nro5'])
df_dates = series_dates.groupby(['Years', 'Month', 'Data'])['value'].value_counts().unstack(fill_value=0)
cols = ['Nro1', 'Nro2', 'Nro3', 'Nro4', 'Nro5']
month_history = df_dates[df_dates.index.get_level_values('Data').isin(cols)].groupby(level=['Years', 'Month']).sum()

In [ ]:
COMBINATIONS = [(3,2), (2,3), (1,4), (4,1), (0,5), (5,0)]
low_high, odd_even = data_functions.combination_count(db, total_numbers, winning_numbers)
low_high_df, odd_even_df = data_functions.combination_df(db, low_high, odd_even)
low_high = pd.DataFrame(low_high)
odd_even = pd.DataFrame(odd_even)
LHOE = pd.concat([low_high, odd_even], axis=1)
LHOE.columns = ['L', 'H', 'O', 'E']
low_high = LHOE.iloc[:, 0:2]
odd_even = LHOE.iloc[:, 2:4]

In [ ]:
last_year = db['Dates'].iloc[-1]
current_year = last_year.year
year_info = numbers_year_history.loc[['Median', 'Average', current_year], :50]
year_criteria = {key: [] for key in total_numbers}
df_median = year_info.loc['Median']
df_average = year_info.loc['Average']

for number in total_numbers:
    x = year_info.at[current_year, number]
    median_half = df_median[number] / 2

    if x == 0 or x <= median_half:
        y = round((1 - ((df_median[number] * 100) / df_average[number]) / 100), 2)
    else:
        x_percentage = round((x * 100 / df_average[number]) / 100, 2)
        y = round((1 - x_percentage if x_percentage > 1 else 1 - x_percentage), 2)

    if np.isnan(y):
        y = 0

    year_criteria[number] = float(y)
    
year_criteria = pd.DataFrame.from_dict(year_criteria, orient='index', columns=['year_criteria'])

In [ ]:
# Finding missing numbers
all_numbers = np.arange(1, 51)
best_numbers = df_aprox.loc[df_aprox['Hits'] > df_aprox['Hits_Needed']].index.to_numpy()
normal_numbers = df_exact.loc[(df_exact['Hits'] > df_exact['Hits_Needed']) & ~(df_exact.index.isin(best_numbers))].index.to_numpy()
missing_numbers = np.setdiff1d(all_numbers, np.concatenate((best_numbers, normal_numbers)))

# Rotation information
rotation_info = {number: 0 for number in all_numbers}
for number in best_numbers:
    rotation_info[number] = 2
for number in normal_numbers:
    rotation_info[number] = 1

# Finding current hits needed
current_hits_needed = data_functions.minimal_hits(db, total_hits, all_numbers, numbers_average, aprox=True)

# Finding rotation hit
draw_until_rotation = len(db['Sorteo']) * data_functions.minimal_hits(db, total_hits, all_numbers, numbers_average, aprox=True) / len(db)
rotation_hit = next((draw for draw in range(len(db['Sorteo'])+1, len(db['Sorteo'])+12) if draw * data_functions.minimal_hits(db, total_hits, all_numbers, numbers_average, aprox=True) / len(db) > int(current_hits_needed) + 1), None)

# Rotation criteria
rotation_criteria = {}
for i in all_numbers:
    category = rotation_info[i]
    diff_exact = df_exact.at[i, 'Difference']
    diff_aprox = df_aprox.at[i, 'Difference']
    hits_exact = df_exact.at[i, 'Hits']
    if category == 1 and diff_exact > 0.50:
        rotation_criteria[i] = 0.50
    elif category == 1 and -1 < diff_exact <= 0.40:
        x = (abs(diff_exact) + (Decimal('0.5') * Decimal('100')) / Decimal('0.50')) / Decimal('100') + Decimal('0.50')
        rotation_criteria[i] = float(x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP))
    elif category == 2 and hits_exact > current_hits_needed:
        rotation_criteria[i] = 1
    elif category == 2 and diff_aprox > -1 and rotation_hit is not None and rotation_hit - len(db) < 4 and len(best_numbers) < 17:
        x = (abs(diff_exact) * Decimal('100')) / Decimal('1.00') / Decimal('100') + Decimal('1')
        rotation_criteria[i] = float(x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP))
    else:
        rotation_criteria[i] = 0.25

rotation_criteria = pd.DataFrame.from_dict(rotation_criteria, orient='index', columns=['rotation_criteria'])

In [ ]:
future_groups_df.columns = [1,2,3,4,5]
gp10 = {}
gp_values = {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 0.70, 6: 0.675, 7: 0.650, 8: 0.625, 9: 0.6, 10: 0.5}

for column, gp in future_groups_df.loc['10_games'].items():
    gp_range = tuple(range(1 + 10*(column-1), 11 + 10*(column-1)))
    gp10[gp_range] = gp_values.get(gp, 0)

gp10_df = pd.DataFrame.from_dict(gp10, orient='index').T
gp10_df.columns = [1,2,3,4,5]
gp10_df = gp10_df.rename(index={0: 'Criteria'})

group_criteria_dict = {key:[] for key in total_numbers}
for i, group in enumerate(groups):
    gn = group
    for number in total_numbers:
        if number in gn:
            group_criteria_dict[number] = gp10_df.iloc[0,i]

index_labels = range(1,51)
group_criteria = pd.DataFrame.from_dict(group_criteria_dict, orient='index', columns=['group_criteria'])

In [ ]:
skips_7_12[['values_7', 'values_12']] = skips_7_12[['7', '12']].applymap(lambda x: data_functions.games_7(x) if x <= 7 else data_functions.games_12(x))

position_criteria = {key: [] for key in total_numbers}

game0 = 0.5
if skips_7_12.iloc[0, 0] <= 5:
    skips_7_12.iloc[0, 2] = game0

for i, key in zip(last_draw.iloc[:,1], last_draw.iloc[:,0]):
    value_added = False
    for e in range(len(skips_7_12)):
        if skips_7_12.iloc[e, 0] <= 7 and e == i:
            position_criteria[int(key)] = skips_7_12.at[e, 'values_7']
            value_added = True
        elif skips_7_12.iloc[e, 0] >= 8 and e == i:
            position_criteria[int(key)] = skips_7_12.at[e, 'values_12']
            value_added = True
    if not value_added:
        if len(last_draw.loc[last_draw['Skips'] <= 12]) <= 38:
            position_criteria[int(key)] = 1
        else:
            position_criteria[int(key)] = 0.25
            
position_criteria = pd.DataFrame.from_dict(position_criteria, orient='index', columns=['position_criteria'])

In [ ]:
df_numbers = pd.concat([year_criteria, rotation_criteria, group_criteria, position_criteria], axis=1)
df_numbers['Criteria'] = df_numbers.apply(lambda row: row['year_criteria'] + row['rotation_criteria'] + row['group_criteria'] + row['position_criteria'], axis=1)
tomorrow_numbers = df_numbers[['Criteria']]
tomorrow_numbers = tomorrow_numbers.sort_values(by='Criteria', ascending=False)
criterion = tomorrow_numbers['Criteria'].median()
recommended_numbers = tomorrow_numbers.loc[tomorrow_numbers['Criteria'] >= criterion].reset_index().rename(columns={'index': 'Numbers'})
not_recommended_numbers = tomorrow_numbers.loc[tomorrow_numbers['Criteria'] < criterion].reset_index().rename(columns={'index': 'Numbers'})